<a href="https://www.kaggle.com/code/abhijitdarekar001/fine-tuning-gemma-python-code-instruction-and-qa?scriptVersionId=165514795" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Table of Contents:
 
1. Installing Libraries & Loading Libraries
2. Loading and Pre-Processing Data<br>
3. Loading Gemma <br>
   a. What should be the value of target modules?<br>
   b. Initilizing LoRA<br>
   c. Training Model<br>
   d. Saving Model<br>
   e. Load and Merge Model Weights<br> 
4. Text Generation Using LoRA Model
5. Conclusion


#### Dataset Used
- [Glaive Python Code QA Dataset](https://www.kaggle.com/datasets/thedevastator/glaive-python-code-qa-dataset): This dataset has  common questions about the Python programming language and other programming languages also.
- [Python Code Instruction](https://www.kaggle.com/datasets/thedevastator/python-code-instruction-dataset) : Dataset consisting of corresponding instruction, input, output, and prompt information of Python Programming Language.


# 1. Installing and Loading Libraries

In [ ]:
!pip install -q -U peft evaluate transformers==4.38.0 accelerate==0.27.2 bitsandbytes==0.42.0 
!pip  install -U -q trl==0.7.10 datasets==2.17.1
# %pip  install -q -U rouge_score

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments,EvalPrediction,pipeline
from peft import LoraConfig, TaskType, get_peft_model , PeftModel, LoraModel
from accelerate.utils import release_memory
from accelerate import Accelerator
from datasets import Dataset
import evaluate
import accelerate

from sklearn.metrics import f1_score
from typing import Dict

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from trl import SFTTrainer

import torch
import torch.nn

import os
os.environ['TOKENIZERS_PARALLELISM'] = "FALSE"
import warnings
warnings.filterwarnings("ignore")
import wandb
import gc
gc.collect()

2024-03-05 06:33:42.907206: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 06:33:42.907259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 06:33:42.908688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


60

In [3]:
wandb.init() 

 # 2. Loading & Pre-Processing Data

In [4]:
code = pd.read_csv("/kaggle/input/python-code-instruction-dataset/train.csv")
code = code.sample(frac = 0.1,random_state = 101,ignore_index=True)
qa = pd.read_csv("/kaggle/input/glaive-python-code-qa-dataset/train.csv",names = ['output','instruction'])
qa = qa.sample(frac = 0.01,random_state = 101,ignore_index=True)

code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1861 entries, 0 to 1860
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  1861 non-null   object
 1   input        1307 non-null   object
 2   output       1861 non-null   object
 3   prompt       1861 non-null   object
dtypes: object(4)
memory usage: 58.3+ KB


In [5]:
def preprocess_data(example):
    template = f"""Instruction:\n{example['instruction']}\n\nResponse:\n{example['output']}"""
    return template
    
code['template'] = code.apply(preprocess_data,axis=1)
qa['template'] = qa.apply(preprocess_data,axis=1)

dataset = Dataset.from_pandas(pd.concat([code[['template']],qa[['template']]])).train_test_split(test_size = 0.2)

# Example Printing
print(code['template'].iloc[1])

Instruction:
Write a Python script to print all prime numbers between two numbers, a and b (where a<b).

Response:
for num in range(a, b+1): 
   if num > 1: 
       for i in range(2, num): 
           if (num % i) == 0: 
               break
       else: 
           print(num)


Above is a sample template from created dataset.

# 3. Loading Gemma

#### What should be the value of target modules?

In [6]:
model_structure = """This is the structure of Gemma Model,


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=256000, bias=False)
)"""
print(model_structure)

This is the structure of Gemma Model,


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_at

When you print model Summary, you will see someting like **Model Attention Layer** in this case you see **GemmaSdpaAttention** which is attention layer of the model.<br>
In our case we have attention layers which are `q_proj`, `k_proj`, `v_proj`, `o_proj`, `rotary_emb`, `up_proj`, `gate_proj`, `down_proj`.<br>
Refer : [target-modules-for-applying-peft-lora-on-different-models](https://stackoverflow.com/questions/76768226/target-modules-for-applying-peft-lora-on-different-models)


In case of LoRA, we have to mention target_modules when defining config for taining a model.
Here are possible values of `target_modeuls` 
1. `target_modules = "all-linear"`<br>
    This is a convenient shortcut that tells the system to target all linear layers within the pre-trained model for adaptation. This includes the modules you saw previously like `q_proj`, `k_proj`, `v_proj`, `o_proj`,`up_proj`, and `down_proj`.<br>
    Apart from these, Embedding layers and Output projection layer,
    
     - Benfits :<br>
     a. Simplicity  : it is sraightforward way to adapt a large portion of the model, potentially achieving significant memory savings.<br>
     b. Potential Performance Gains : As we are including majority of the model, this can help model to learn more efficintly which leads to better fine-tuning performanc.
     
     - Cons :<br>
     a. Over-adaptation : Adapting all linear layers can be overly aggressive, potentially leading to a loss of information from the pre-trained model and hindering performance. <br>
     b. Increased Memory Foot print : While aiming for memory efficiency, adapting all linear layers will still require some memory overhead compared to targeting specific modules.<br>
     
2. `target_modeules = ["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "gate_proj", "down_proj"]`<br>
    To over come the over-fitting we can secifying individual modules, you can explicitly list the modules you want to adapt for more control. This allows you to focus on specific areas while leaving less relevant parts untouched.<br>
    Below is the modules desciption.<br>
     - `q_proj`,`k_porj`,`v_proj` are query, key, values in attention mechinism.
     - `gate_proj` Projection layer for the gate in the attention mechanism.
     - `up_proj`, `down_proj` are Projection layers used for the feed-forward network within a transformer block.
    

## Initilizing LoRA

In [7]:
base_model = "/kaggle/input/gemma/transformers/2b-it/2"
lora_config = LoraConfig(
        lora_dropout=0.1,
        r=8,
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='fp4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config)

loraModel = LoraModel(model= model,config =lora_config,adapter_name = "adapter")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##### Question : What is number of  trainable parameters in Model?

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(loraModel)

trainable params: 9805824 || all params: 1525073920 || trainable%: 0.6429736861541767


In [9]:
model_output_dir = "gemma2b"
training_args = TrainingArguments(
    output_dir=model_output_dir,                   
    num_train_epochs=1,                       
    per_device_train_batch_size=1,           
    gradient_accumulation_steps=8,      
    per_device_eval_batch_size = 1,
    gradient_checkpointing=True,           
    logging_steps=25,                        
    learning_rate=1e-4,                       
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                    
    max_steps=-1,
    warmup_ratio=0.03,                        
    group_by_length=True,
    lr_scheduler_type="cosine",                             
    evaluation_strategy="steps",
    report_to="wandb"
    )


trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    peft_config=lora_config,
    tokenizer=tokenizer,
    max_seq_length=256,
    dataset_text_field = 'template',
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/2577 [00:00<?, ? examples/s]

Map:   0%|          | 0/645 [00:00<?, ? examples/s]


## Training Model

In [10]:
torch.cuda.empty_cache()
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
25,1.930200,1.228644
50,1.029400,1.093736
75,1.130400,1.017548
100,0.863100,0.998809
125,0.993500,0.984391
150,0.856300,0.963941
175,0.941900,0.961933
200,0.837300,0.947427
225,0.960500,0.940101
250,0.851300,0.932984


TrainOutput(global_step=322, training_loss=1.0088226780387926, metrics={'train_runtime': 4408.2414, 'train_samples_per_second': 0.585, 'train_steps_per_second': 0.073, 'total_flos': 5511819935846400.0, 'train_loss': 1.0088226780387926, 'epoch': 1.0})

## Saving Adapter

In [11]:
lora_adapter_path = "LoraAdapter"
trainer.model.save_pretrained(lora_adapter_path)

In [12]:
# Release Memory
trainer, model, = release_memory(trainer, model)
gc.collect()
torch.cuda.empty_cache()


## Load and Merge Model Weights 

In [13]:
final_model = "Final_Model"
# Loading Base Model
model = AutoModelForCausalLM.from_pretrained(base_model,device_map='auto', torch_dtype=torch.float16)

# Lodaing  Base Model with LoRA adapters.
peft_model = PeftModel.from_pretrained(model,lora_adapter_path,device_map='auto', torch_dtype=torch.float16)

# Merging and Saving Model
model = peft_model.merge_and_unload(progressbar = True)
model.save_pretrained(final_model)
tokenizer.save_pretrained(final_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unloading and merging model: 100%|██████████| 384/384 [00:00<00:00, 4890.31it/s]


('Final_Model/tokenizer_config.json',
 'Final_Model/special_tokens_map.json',
 'Final_Model/tokenizer.model',
 'Final_Model/added_tokens.json',
 'Final_Model/tokenizer.json')

In [14]:
model = release_memory(model)

# 4. Text Generation Using LoRA Model

In [15]:
peft_pipe = pipeline("text-generation",final_model, model_kwargs={"torch_dtype": torch.float16},
    device_map='auto',
    max_new_tokens=512)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
def get_output(question):
    prompt = f"Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.\n\nInstruction:\n{question}\n\nResponse:\n"
    out = peft_pipe(prompt,
     do_sample=True,
    temperature=0.1,
    top_k=20,
    top_p=0.3,
    add_special_tokens=True)
                    
    return out

In [18]:
out = get_output("Write program to find factorial of number")
print(out[0]['generated_text'])

Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.

Instruction:
Write program to find factorial of number

Response:
```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```


In [22]:
out = get_output("Explain how can an AI classify a given sentence as a declaration, an expression, or a statement?")
print(out[0]['generated_text'])

Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.

Instruction:
Explain how can an AI classify a given sentence as a declaration, an expression, or a statement?

Response:
An AI can classify a given sentence as a declaration, an expression, or a statement by analyzing the context and the presence of certain keywords and grammatical structures.


In [23]:
out = get_output("I need to write a bash script that compares two arguments and prints a message if the first argument is greater than the second argument. Can someone help me with the code")
print(out[0]['generated_text'])

Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.

Instruction:
I need to write a bash script that compares two arguments and prints a message if the first argument is greater than the second argument. Can someone help me with the code

Response:
```bash
#!/bin/bash

if [[ "$1" > "$2" ]]; then
  echo "The first argument is greater than the second argument."
fi
```


In [24]:
out = get_output("How can I optimize a TensorFlow neural network to improve its predictive accuracy?")
print(out[0]['generated_text'])

Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.

Instruction:
How can I optimize a TensorFlow neural network to improve its predictive accuracy?

Response:
1. **Data Preparation:** Clean and normalize the data to ensure consistent representation.
2. **Model Optimization:** Choose an appropriate model architecture and hyperparameters to achieve optimal performance.
3. **Training and Evaluation:** Implement a robust training loop with early stopping and validation to prevent overfitting.
4. **Regularization:** Introduce regularization techniques to prevent overfitting and improvegeneralizability.
5. **Feature Engineering:** Explore feature engineering techniques to extract more relevant features from the data.
6. **Hyperparameter Tuning:** Optimize the model's hyperparameters to find the best settings for the task.
7. **Ensemble Methods:** Combine multiple models to leverage their strengths and i

# 5. Conclusion 

In this notebook we have fine-tuned **Gemma-2b-it** model from **HuggingFace**  for Python-Code Generation.

We have covered following topics.
- Dataset preprocessing.
- Defining Lora-config for model. 
- Why Choosing correct target Modules are nesseary?.
- Saving and Merging Lora Adapters with Model.
- Text Generation for Lora Mode


# Thank you.
### Please do suggest if there is any corrections related to approach,design, preprocessing and model building.